In [4]:
import pickle
import pandas as pd

with open('data_pickle_format/synethetic_vacancies_final.pickle', 'rb') as f:
    df = pickle.load(f)   
    
df['title'] = df['job_description'].str.extract(
    r'(?:Job\s*Title|Title):\s*(.*?)(?:\.|\n)\s*(?:Job\s*Description|Description):', 
    expand=False
)
df['description'] = df['job_description'].str.extract(r'(?:Job\s*Description|Description):\s*(.*)', expand=False)

In [5]:
import spacy

nlpNL = spacy.load("nl_core_news_sm")
nlpEN = spacy.load("en_core_web_sm")

filter_and_lemma = []

for text in df['description']:
    if pd.notnull(text): 
        doc = nlpEN(text)
        filter_and_lemma.append([token.lemma_ for token in doc if not (token.is_stop or token.is_punct)])
    else:
        filter_and_lemma.append([])  
df['lemmas'] = filter_and_lemma

In [6]:
split_titles = []
for text in df['title']:
    if pd.notnull(text): 
        doc = nlpEN(text)
        split_titles.append([token.lower_ for token in doc if not (token.is_stop or token.is_punct)])
    else:
        split_titles.append([])  
df['split'] = split_titles
df['split']

0                 [support, technician]
1                     [data, scientist]
2      [information, security, manager]
3           [people, analyst, hr, tech]
4                [elearning, developer]
                     ...               
361         [ict, security, specialist]
362      [telecommunications, engineer]
363           [senior, data, scientist]
364                   [erp, consultant]
365               [software, developer]
Name: split, Length: 366, dtype: object

In [7]:
with open('data_pickle_format/labeled_vacancies_final.pickle', 'rb') as f:
    real_df = pickle.load(f)

from string import punctuation
import html
import re

def preprocess_text(text):
    text = text.lower()
    text = html.unescape(text)
    text = re.sub(r"https?://\S+", "", text) # remove http links
    text = re.sub(r"<.*?>", " ", text) # remove html tags
    text = re.sub(f"[{re.escape(punctuation)}]", "", text)  # Remove punctuation
    text = " ".join(text.split())  # Remove extra spaces, tabs, and new lines
    return text

real_df["description"] = real_df["description"].map(preprocess_text)

pd.set_option('display.max_colwidth', None)


In [8]:

import py3langid as langid

languages = []
for description in real_df['description']:
    lang, confidence = langid.classify(description)
    languages.append(lang)

real_df['language'] = languages

In [9]:
known_titles = set(" ".join(tokens) for tokens in df['split'] if tokens)

def extract_known_title(text, known_titles, nlp):
    if pd.isnull(text):
        return None
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    text_norm = " ".join(tokens)
    
    for title in known_titles:
        if title in text_norm:
            return title
    return None


real_df.loc[real_df['language'] == 'nl', 'extracted title'] = real_df.loc[real_df['language'] == 'nl', 'description'].apply(lambda x: extract_known_title(x, known_titles, nlpNL))
real_df.loc[real_df['language'] == 'en', 'extracted title'] = real_df.loc[real_df['language'] == 'en', 'description'].apply(lambda x: extract_known_title(x, known_titles, nlpEN))                                                                                                                            

In [12]:
len(real_df.loc[real_df['extracted title'].notnull()])

653

In [13]:
len(real_df.loc[real_df['extracted title'].isnull()])

1722

In [15]:
from rapidfuzz import fuzz, process

def fuzzy_extract_title(text, known_titles, nlp, threshold=85):
    if pd.isnull(text):
        return None
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    text_norm = " ".join(tokens)
    
    match, score, _ = process.extractOne(text_norm, known_titles, scorer=fuzz.partial_ratio)
    return match if score > threshold else None

real_df.loc[real_df['language'] == 'nl', 'fuzzy title'] = real_df.loc[real_df['language'] == 'nl', 'description'].apply(lambda x: fuzzy_extract_title(x, known_titles, nlpNL))
real_df.loc[real_df['language'] == 'en', 'fuzzy title'] = real_df.loc[real_df['language'] == 'en', 'description'].apply(lambda x: fuzzy_extract_title(x, known_titles, nlpEN))                                                                                                                            

In [16]:
len(real_df.loc[real_df['fuzzy title'].notnull()])

1662

In [17]:
len(real_df.loc[real_df['fuzzy title'].isnull()])

713

In [21]:
real_df.loc[real_df['language']=='en'].tail(5)

description  \
2312                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            company name bekend bij yacht job requirements you have knowledge and experience in executing infrastructure projects and outsourcing processes you can quickly navigate a large international organization with various stakeholders you are familiar with prince2 devops agile certification is not a requirement but your cv and portfolio of work demonstrate your proficiency in these areas as a candidate you do have experience in transition project management preferably with companies such as dxchl deep technical knowledge is not essential but a certain way of conceptual thinking for networking more specifically wlan connection good command of the english language dutch is considered as an advantage living in the nethe

In [26]:
real_df.rename(columns={'fuzzy title' : 'positiontitle', 'description': 'job_description'}, inplace=True)
real_df[['positiontitle','job_description', 'women_proportion']]

,positiontitle,job_description,women_proportion
0,trainer,company name ministerie van financi n belastingdienst job requirements tmap next foundation ervaring met projecten in een agile omgeving senior tester met ervaring in testautomatisering 5 jaar kennis en ervaring met agilescrum wensen testtooling robotframework functioneel testen integratietesten performancetesten kennis van xml mq db2 kennis van container platforms location apeldoorn nederland none company info none compensation benefits dit is een opdracht tussen de 32 36 uur per week de startdatum is in overleg en de looptijd van de opdracht is in eerste instantie tot 31 december 2023 waarna er optie tot verlenging is voor deze rol zal je door yacht gedetacheerd worden bij de belastingdienst als senior tester je reiskosten worden uiteraard volledig vergoed een elektrische leaseauto welke je ook priv mag rijden behoort tot de mogelijkheden ook ontvang je een bonusregeling tot maximaal 8 van je bruto jaarsalaris 25 vakantiedagen en 833 vakantiegeld op fulltime jaarbasis daarnaast gaan we graag met je in gesprek over je persoonlijke ontwikkelingsplan en bespreken welke trainingen en opleidingen gewenst zijn want blijven ontwikkelen zijn de sleutelwoorden job type tijdelijke functieopdracht certifications tmap next foundation ervaring met projecten in een agile omgeving senior tester met ervaring in testautomatisering 5 jaar kennis en ervaring met agilescrum wensen testtooling robotframework functioneel testen integratietesten performancetesten kennis van xml mq db2 kennis van container platforms remote possible false,0.400000
1,scrum master,company name belastingdienst job requirements gecertificeerd scrum master minimaal psm ii of vergelijkbaar meerdere jaren aantoonbare ervaring als scrum master waaronder ervaring met safe of andere opschalingsmodellen ervaring met implementatie trajecten binnen een complexe omgeving professioneel vaardig in het nederlands en engels location apeldoorn nederland none company info de iv organisatie van de belastingdienst is verantwoordelijk voor en verzorgt de ict voorzieningen het merendeel van de applicaties wordt op dit moment door de iv organisatie zelf ontwikkeld onderhouden en beheerd in het eigen data center naast de zorg voor continu teit op de massale heffing en inningsprocessen die plaatsvinden binnen een degelijke stabiele omgeving wordt er tevens volop gewerkt aan modernisering van het iv landschap dit gebeurt deels intern door gebruik te maken van de expertise die intern aanwezig is maar ook door het aantrekken van kantenklaar oplossingen en expertise uit de markt compensation benefits auto van de zaak is bespreekbaar en mag uiteraard prive gebruikt worden salaris op basis van kennis en ervaring met een maximum van 5040 per maand indien gewenst direct vast contract voor onbepaalde tijd bij yacht voor deze opdracht geldt een 13e maand en voor andere opdrachten een 13e maand of deelname aan de yacht bonusregeling 833 vakantiegeld 25 vakantiedagen als je belooft dat je er iets leuks mee doet mogelijkheid voor het volgen van opleidingen cursussen trainingen pluspensioen vanaf de eerste werkdag bij ziekte eerste jaar 100 doorbetaald job type tijdelijke functieopdracht certifications gecertificeerd scrum master minimaal psm ii of vergelijkbaar meerdere jaren aantoonbare ervaring als scrum master waaronder ervaring met safe of andere opschalingsmodellen ervaring met implementatie trajecten binnen een complexe omgeving professioneel vaardig in het nederlands en engels remote possible true,0.750000
2,None,company name duo job requirements 3 jaar ervaring in het volgende eisenpakket hbo denk en werkniveau relevante werkervaring in modelleren relevante werkervaring in een java omgeving relevante werkervaring met rest nice to have ervaring met java swing is een pr ervaring met velocity is een pr location groningen none company info compensation benefits deze opdracht staat ook open voor doorleen en zzpers job type tijdelijke functieopdr

In [27]:
real_df[['positiontitle','job_description', 'women_proportion']].to_pickle("./data_pickle_format/titlelabeled.pickle")